<a href="https://colab.research.google.com/github/dinh-thang/COS30018-Project-C/blob/main/Falcon_1b_with_Biomed_qa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Full-finetuning Distilled Bert on Covid_qa_deepset dataset**

Load and install necessary libraries and dependancies

In [2]:
!pip install -q transformers einops
!pip install transformers[torch]
!pip install datasets wandb
!pip install --upgrade transformers
!pip install evaluate numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 48.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 87.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 66.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 48.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

# Load the dataset

Start by loading the Covid_qa_deepset dataset from Huggingface. 70% of the dataset is extracted for training. 303 rows (not overlap with the training set) will be used for validation set

In [3]:
from datasets import load_dataset

dataset_name = "covid_qa_deepset"
dataset = load_dataset(dataset_name, split="train[:1969]")
dataset = dataset["train"].train_test_split(train_size=1413,test_size=303)

dataset = dataset.filter(lambda example: not "answer_category" in example['answers'])

Generating train split:   0%|          | 0/2019 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1413 [00:00<?, ? examples/s]

Filter:   0%|          | 0/303 [00:00<?, ? examples/s]

Load the tokenizer from the model

In [4]:
from transformers import AutoTokenizer

# load distilled bert tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased-distilled-squad")

Preprocess the dataset based on the guideline provided by Huggingface

In [ ]:
def preprocess_training(example):
    inputs = tokenizer(
        example['question'],
        example["context"],
        max_length=512,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = example["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

def preprocess_eval(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=512,
        truncation="only_second",
        stride=128,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs

training_dataset = dataset['train']
training_dataset = training_dataset.map(preprocess_training, batched=True, remove_columns=dataset['train'].column_names)

eval_dataset = dataset['test']
eval_dataset = eval_dataset.map(preprocess_training, batched=True, remove_columns=dataset['test'].column_names)


The last 50 rows of the dataset will be used for testing, which is unseen by model. This is to give unbiased evaluation for the model's performance

In [5]:
test_dataset = load_dataset(dataset_name, split="train[1969:]") #test dataset is extracted from last 50 rows of CovidQA dataset

# Check the format of datasets after preprocess



In [ ]:
training_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'start_positions', 'end_positions'],
    num_rows: 1413
})

In [ ]:
eval_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'start_positions', 'end_positions'],
    num_rows: 303
})

In [ ]:
test_dataset

Dataset({
    features: ['document_id', 'context', 'question', 'is_impossible', 'id', 'answers'],
    num_rows: 50
})

# Load the model

The model used is a variant of Bert, with 40% less parameters than typical Bert. This model is also finetuned with SQuAD dataset before.

In [ ]:
import torch
import transformers
from transformers import DistilBertTokenizer, DistilBertForQuestionAnswering

model_name = "distilbert-base-uncased-distilled-squad"


model = DistilBertForQuestionAnswering.from_pretrained(
    model_name,
)
model.config.use_cache = False


# Start the training

Set up data collator for data enumeration

In [ ]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

Configure the training arguments

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="Distilled_bert_CovidQA_model",
    evaluation_strategy = "steps",
    save_steps = 10,
    logging_steps = 10,
    max_steps = 500,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,
)


In [ ]:
from transformers import Trainer

max_seq_length = 2048

trainer = Trainer(
    model=model,
    train_dataset=training_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    args=training_args,
    data_collator = data_collator
)

Log in to wandb to configure training loss visualisation and save the monitor

In [ ]:
import wandb
wandb.login(key="2ba90e109c0da4331467290174e54e3483f0d494")
wandb.init(project="QLoRA covidQA")

In [ ]:
trainer.train()


Step,Training Loss,Validation Loss,Accuracy
10,1.126500,0.906048,0.801980
20,0.609300,0.718573,0.841584
30,0.658100,0.718396,0.838284
40,0.871600,0.726818,0.828383
50,0.819900,0.686123,0.864686
60,0.920200,0.678616,0.851485
70,0.664600,0.650183,0.851485
80,0.549400,0.663274,0.848185
90,0.763000,0.688111,0.844884
100,0.509100,0.664566,0.854785


TrainOutput(global_step=500, training_loss=0.38069887399673463, metrics={'train_runtime': 9669.35, 'train_samples_per_second': 0.827, 'train_steps_per_second': 0.052, 'total_flos': 1038038879754240.0, 'train_loss': 0.38069887399673463, 'epoch': 5.62})

#Run evaluation

This is evaluation on test dataset, which has not been seen by the model. This is the evaluation of the finetuned Distilled Bert

In [ ]:
#evaluation with test dataset (unseen by model)
from transformers import pipeline

pipeline = pipeline("question-answering",model="distilbert-base-uncased-distilled-squad", tokenizer=tokenizer)

total_f1 = 0
total_accuracy = 0

for ins in test_dataset:
  ans = pipeline(question=ins['question'], context=ins['context'], max_answer_len=50, max_question_len=300)
  ref_tokens = tokenizer(" " + ins["answers"]["text"][0])["input_ids"]
  ans_tokens = tokenizer(ans["answer"])["input_ids"]
  common_tokens = set(ans_tokens) & set(ref_tokens)
  precision = len(common_tokens) / len(ans_tokens)
  recall = len(common_tokens) / len(ref_tokens)
  total_accuracy += precision
  #print(tokenizer.decode(ans_tokens), "|", tokenizer.decode(ref_tokens), "|")
  if (len(common_tokens) == 0):
    total_f1 += 0
  else:
    f1 = 2 * precision * recall / (precision + recall)
    total_f1 += f1

In [ ]:
print("F1 average score:", total_f1 / 50)
print("Accuracy average score: ", total_accuracy / 50)

F1 average score: 0.633776704091313
Accuracy average score:  0.7655031439244173


This is the evaluation of Distilled Bert before finetuning

In [6]:
#evaluation with test dataset (unseen by model)
from transformers import pipeline

pipeline = pipeline("question-answering",model="distilbert-base-uncased-distilled-squad", tokenizer=tokenizer)

total_f1 = 0
total_accuracy = 0

for ins in test_dataset:
  ans = pipeline(question=ins['question'], context=ins['context'], max_answer_len=50, max_question_len=300)
  ref_tokens = tokenizer(" " + ins["answers"]["text"][0])["input_ids"]
  ans_tokens = tokenizer(ans["answer"])["input_ids"]
  common_tokens = set(ans_tokens) & set(ref_tokens)
  precision = len(common_tokens) / len(ans_tokens)
  recall = len(common_tokens) / len(ref_tokens)
  total_accuracy += precision
  #print(tokenizer.decode(ans_tokens), "|", tokenizer.decode(ref_tokens), "|")
  if (len(common_tokens) == 0):
    total_f1 += 0
  else:
    f1 = 2 * precision * recall / (precision + recall)
    total_f1 += f1

In [7]:
print("F1 average score:", total_f1 / 50)
print("Accuracy average score: ", total_accuracy / 50)

F1 average score: 0.46529701849430466
Accuracy average score:  0.7257254317367219


# Inference

Test inference for the model before and after finetuning

In [ ]:
# Before training
from transformers import pipeline, QuestionAnsweringPipeline

question = "Where did Covid-19 originated"
context = 'According to epidemiological studies, the Huanan Market in Wuhan was the early and main epicentre of SARS‐CoV‐2 infection'
print(question)

pipe = pipeline(model="distilbert-base-uncased-distilled-squad")
pipe(question=question, context=context, max_answer_len=300, max_question_len=300)

Where did Covid-19 originated


{'score': 0.35626348853111267,
 'start': 42,
 'end': 64,
 'answer': 'Huanan Market in Wuhan'}

In [ ]:
#After training
question = "Where did Covid-19 originated"
context = "According to epidemiological studies, the Huanan Market in Wuhan was the early and main epicentre of SARS‐CoV‐2 infection"
print(question)
#print(context)

pipe2 = pipeline(model="goodcoffee/Distilled_bert_CovidQA_model", tokenizer = tokenizer)
pipe2(question=question, context=context, max_answer_len=300, max_question_len=300)

Where did Covid-19 originated


{'score': 0.4766525328159332,
 'start': 42,
 'end': 64,
 'answer': 'Huanan Market in Wuhan'}